In [322]:
%matplotlib inline

import matplotlib as mpl
# mpl.use('TkAgg')
# import matplotlib.cm as cm
# from scipy.ndimage import filters as filters
from skimage import io, color, util
import numpy as np
# import math
import os
# import utils as custom_utils
import collections as coll

In [323]:
images = ['Images/Q3/castle.jpg']

In [324]:
image = io.imread(images[0])
# image = color.rgb2lab(image)

In [367]:
height, width = image.shape[:2]
n_segments = 100
max_iter = 10
spatial_weight = 1

In [357]:
def get_k_centers(k, height, width):
    grid_y, grid_x = np.mgrid[:height, :width]
    slices = util.regular_grid((height, width), k)
    step_y, step_x = [int(s.step if s.step is not None else 1) for s in slices]
    segments_y = grid_y[slices]
    segments_x = grid_x[slices]
    
    centers = np.concatenate([segments_y[..., np.newaxis], segments_x[..., np.newaxis]], axis=-1)
    centers = segments_xy.reshape(-1, 2)
    
    return centers, step_y, step_x

In [358]:
segments_xy, step_y, step_x = get_k_centers(n_segments, height, width)

In [359]:
def get_window_bounds(center, step_y, step_x, height, width):
    cy = center[0]
    cx = center[1]

    y_min = max(cy - step_y, 0)
    y_max = min(cy + step_y + 1, height)
    x_min = max(cx - step_x, 0)
    x_max = min(cx + step_x + 1, width)
    return y_min, y_max, x_min, x_max

In [368]:
n_centers = segments_xy.shape[0]

for i in xrange(max_iter):
    change = 0
    for k in xrange(n_centers):
        c = segments_xy[k]
        y_min, y_max, x_min, x_max = get_window_bounds(c, step_y, step_x, height, width)
        for y in xrange(y_min, y_max):
            dy = (c[0] - y) ** 2
            for x in xrange(x_min, x_max):
                dist_center = (dy + (c[1] - x) ** 2) * spatial_weight
                

In [306]:
segments_color = np.zeros(segments_x.shape + (image.shape[2],))
segments_color = segments_color.reshape(-1, image.shape[2])

# print np.concatenate([segments_color, segments_xy], axis=-1)

In [307]:
# compactness=10.
# ratio = 1.0 / compactness